# ML task for students level dataset (regression and classification)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('zno_preprocessed.csv')

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (19,28,37,46,55,64,73,81,90,99,106,107,108,109,111,112,113,114) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# get only one year for test (possible years: 2018,2019,2020)
df = df[df['year']==2020]

In [5]:
df =df.drop(columns=['outid']).reset_index(drop=True)

In [6]:
class Preprocessing:
    DATA_TYPES = ['train','test']

    def __init__(self,
                 drop_strange_st_data: bool = True,
                 column_for_onehot: list = [],
                 scale_columns: list = []):

        self.drop_strange_st_data = drop_strange_st_data
        self.column_for_onehot = column_for_onehot
        self.scale_columns = scale_columns

        self.encoder = OneHotEncoder()
        self.scaler = MinMaxScaler()
        self.is_encoder_fit = False
        self.is_scaler_fit = False

    def process(self, 
                df: pd.DataFrame,
                data_type: str = 'train') -> pd.DataFrame:

        if data_type not in self.DATA_TYPES:
            raise ValueError(f'Unexpected data type. Data type must be one of {self.DATA_TYPES}.')
        
        if ((not self.is_encoder_fit) and (self.column_for_onehot)) and data_type == 'test':
            raise ValueError(f'Please, use train dataset first.')
            
        if ((not self.is_scaler_fit) and (self.scale_columns)) and data_type == 'test':
            raise ValueError(f'Please, use train dataset first.')

        if self.drop_strange_st_data:
            df = self._drop_strange_data_students_data(df)
        
        if self.column_for_onehot:
            df = self._one_hot_encoding(
                df = df,
                column_for_onehot = self.column_for_onehot,
                data_type = data_type
            )
            self.is_encoder_fit = True
        
        if self.scale_columns:
            df = self._scaling_columns(
                df = df,
                scale_columns = self.scale_columns,
                data_type = data_type
            )
            self.is_scaler_fit = True

        object_columns = self._get_object_columns(df)
        df,obj2num = self._encode_str_values(
            df = df,
            obj_columns = object_columns
        )

        self.obj2num = obj2num

        return df

    def _one_hot_encoding(self,
                          df: pd.DataFrame,
                          column_for_onehot: list,
                          data_type: str) -> pd.DataFrame:

        if data_type == 'train':
            data = self.encoder.fit_transform(df[column_for_onehot]).toarray()

        else: 
            data = self.encoder.transform(df[column_for_onehot]).toarray()

        df[self.encoder.get_feature_names_out(column_for_onehot)] = data
        df = df.drop(columns = column_for_onehot)

        return  df
    
    def _scaling_columns(self,
                         df: pd.DataFrame,
                         scale_columns: list,
                         data_type: str) -> pd.DataFrame:
        if data_type == 'train':
            df[scale_columns] = self.scaler.fit_transform(df[scale_columns])
        else: 
            df[scale_columns] = self.scaler.transform(df[scale_columns])

        return df
    
    @staticmethod
    def _encode_str_values(df: pd.DataFrame,
                           obj_columns: list) -> pd.DataFrame:
        obj2num = []
        for col in obj_columns:
            obj2num.append({col:{obj: i for i,obj in enumerate(set(df[col]))}})
            cur_num = obj2num[-1]
            df[col] = df[col].map(lambda x:cur_num[col][x])
        
        return (df,obj2num)

    @staticmethod
    def _drop_strange_data_students_data(df: pd.DataFrame) -> pd.DataFrame:
        return df.reset_index(drop=True)
    
    @staticmethod
    def _get_object_columns(df:pd.DataFrame) -> list:
        obj_col = []
        for column in df.columns:
            if df[column].dtype == 'object':
                obj_col.append(column)

        return obj_col

In [15]:
col = ['sextypename', 'regname', 'areaname', 'tername',
       'regtypename', 'tertypename', 'classprofilename', 'classlangname',
       'eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername',
       'eoparent','mathtest', 'mathlang', 'mathteststatus', 'mathball100', 'mathball',
       'mathptname', 'mathptregname', 'mathptareaname', 'mathpttername','age']

# Regression

In [16]:
X = df[col]

In [17]:
X.head()

,sextypename,regname,areaname,tername,regtypename,tertypename,classprofilename,classlangname,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,mathtest,mathlang,mathteststatus,mathball100,mathball,mathptname,mathptregname,mathptareaname,mathpttername,age
0,чоловіча,Донецька область,м.Маріуполь,Приморський район міста,Випускник української школи поточного року,місто,Історичний,російська,"Комунальний заклад ""Маріупольська загальноосві...",середня загальноосвітня школа,Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,Департамент освіти Маріупольської міської ради...,Математика,російська,Не подолав поріг,"0,0",8.0,"Комунальний заклад ""Маріупольська загальноосві...",Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,17
1,чоловіча,Полтавська область,Миргородський район,с.Хомутець,Випускник української школи поточного року,село,Української філології,українська,"Опорний навчальний заклад ""Хомутецька загально...",середня загальноосвітня школа,Полтавська область,Миргородський район,с.Хомутець,Відділ освіти Миргородської райдержадміністрац...,Математика,українська,Зараховано,"155,0",30.0,Миргородська спеціалізована школа І-ІІІ ступен...,Полтавська область,м.Миргород,м.Миргород,17
2,чоловіча,м.Київ,м.Київ,Святошинський район міста,Випускник української школи поточного року,місто,Універсальний,українська,"Товариство з обмеженою відповідальністю ""Центр...",середня загальноосвітня школа,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,"Управління освіти, молоді та спорту Святошинсь...",Математика,українська,Зараховано,"137,0",21.0,Середня загальноосвітня школа №222 Святошинськ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,18
3,жіноча,Львівська область,м.Львів,Галицький район міста,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,Львівський коледж транспортної інфраструктури ...,заклад вищої освіти,Львівська область,м.Львів. Галицький район міста,Галицький район міста,Міністерство освіти і науки України,Математика,українська,Не з’явився,NaN,NaN,Самбірський технікум економіки та інформатики,Львівська область,м.Самбір,м.Самбір,16
4,жіноча,Дніпропетровська область,м.Кам'янське,Південний район міста,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,Кам'янський коледж фізичного виховання,заклад вищої освіти,Дніпропетровська область,м.Кам'янське. Південний район міста,Південний район міста,Міністерство освіти і науки України,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17


In [18]:
X = X.dropna().reset_index(drop=True)

In [22]:
X['mathball100'] = pd.to_numeric(X['mathball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
X.head()

,sextypename,regname,areaname,tername,regtypename,tertypename,classprofilename,classlangname,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,mathtest,mathlang,mathteststatus,mathball100,mathball,mathptname,mathptregname,mathptareaname,mathpttername,age
0,чоловіча,Донецька область,м.Маріуполь,Приморський район міста,Випускник української школи поточного року,місто,Історичний,російська,"Комунальний заклад ""Маріупольська загальноосві...",середня загальноосвітня школа,Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,Департамент освіти Маріупольської міської ради...,Математика,російська,Не подолав поріг,0.0,8.0,"Комунальний заклад ""Маріупольська загальноосві...",Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,17
1,чоловіча,Полтавська область,Миргородський район,с.Хомутець,Випускник української школи поточного року,село,Української філології,українська,"Опорний навчальний заклад ""Хомутецька загально...",середня загальноосвітня школа,Полтавська область,Миргородський район,с.Хомутець,Відділ освіти Миргородської райдержадміністрац...,Математика,українська,Зараховано,155.0,30.0,Миргородська спеціалізована школа І-ІІІ ступен...,Полтавська область,м.Миргород,м.Миргород,17
2,чоловіча,м.Київ,м.Київ,Святошинський район міста,Випускник української школи поточного року,місто,Універсальний,українська,"Товариство з обмеженою відповідальністю ""Центр...",середня загальноосвітня школа,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,"Управління освіти, молоді та спорту Святошинсь...",Математика,українська,Зараховано,137.0,21.0,Середня загальноосвітня школа №222 Святошинськ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,18
3,жіноча,Львівська область,м.Львів,Галицький район міста,Випускник української школи поточного року,місто,Універсальний,українська,Заклад загальної середньої освіти №87 Львівськ...,середня загальноосвітня школа,Львівська область,м.Львів. Галицький район міста,Галицький район міста,Відділ освіти Галицького та Франківського райо...,Математика,українська,Зараховано,121.0,15.0,Середня загальноосвітня школа №84 імені Блажен...,Львівська область,м.Львів. Сихівський район міста,Сихівський район міста,17
4,жіноча,Рівненська область,Сарненський район,с.Любиковичі,Випускник української школи поточного року,село,Української філології,українська,Любиковицька загальноосвітня школа І-ІІІ ступе...,середня загальноосвітня школа,Рівненська область,Сарненський район,с.Любиковичі,"Відділ освіти, молоді та спорту Сарненської ра...",Математика,українська,Зараховано,124.0,16.0,Сарненський педагогічний коледж Рівненського д...,Рівненська область,Сарненський район,м.Сарни,18


In [25]:
train_df,test_df = train_test_split(X,test_size=0.2,random_state=42)

In [27]:
onehotencode_col = ["sextypename", "regname", "regtypename", "eotypename", "eoregname", "mathlang", "mathptregname"]

In [28]:
preprocessor = Preprocessing(
    drop_strange_st_data = False,
    column_for_onehot = onehotencode_col
)

In [29]:
preprocess_train_df = preprocessor.process(
    df = train_df,
    data_type = 'train'
)
preprocess_train_df

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


,areaname,tername,tertypename,classprofilename,classlangname,eoname,eoareaname,eotername,eoparent,mathtest,mathteststatus,mathball100,mathball,mathptname,mathptareaname,mathpttername,age,sextypename_жіноча,sextypename_чоловіча,regname_Івано-Франківська область,regname_Волинська область,regname_Вінницька область,regname_Дніпропетровська область,regname_Донецька область,regname_Житомирська область,regname_Закарпатська область,regname_Запорізька область,regname_Київська область,regname_Кіровоградська область,regname_Луганська область,regname_Львівська область,regname_Миколаївська область,regname_Одеська область,regname_Полтавська область,regname_Рівненська область,regname_Сумська область,regname_Тернопільська область,regname_Харківська область,regname_Херсонська область,regname_Хмельницька область,regname_Черкаська область,regname_Чернівецька область,regname_Чернігівська область,regname_м.Київ,regtypename_Випускник коледжу,regtypename_Випускник української школи поточного року,regtypename_Студент закладу вищої освіти,eotypename_вечірня (змінна) школа,eotypename_вище професійне училище,eotypename_вище художнє професійно-технічне училище,eotypename_гімназія,eotypename_загальноосвітня санаторна школа,eotypename_заклад вищої освіти,eotypename_колегіум,eotypename_коледж,eotypename_колеж,eotypename_ліцей,eotypename_навчально-виховне об'єднання,eotypename_навчально-виховний комплекс,eotypename_навчально-реабілітаційний центр,eotypename_професійний ліцей відповідного профілю,eotypename_професійно-технічне училище відповідного профілю,eotypename_середня загальноосвітня школа,eotypename_середня загальноосвітня школа-інтернат,eotypename_спеціальна загальноосвітня школа,eotypename_спеціальна загальноосвітня школа-інтернат,eotypename_спеціалізована школа,eotypename_спеціалізована школа-інтернат,eotypename_центр професійно-технічної освіти,eotypename_центр професійної освіти,eotypename_центр підготовки і перепідготовки робітничих кадрів,eoregname_Івано-Франківська область,eoregname_Волинська область,eoregname_Вінницька область,eoregname_Дніпропетровська область,eoregname_Донецька область,eoregname_Житомирська область,eoregname_Закарпатська область,eoregname_Запорізька область,eoregname_Київська область,eoregname_Кіровоградська область,eoregname_Луганська область,eoregname_Львівська область,eoregname_Миколаївська область,eoregname_Одеська область,eoregname_Полтавська область,eoregname_Рівненська область,eoregname_Сумська область,eoregname_Тернопільська область,eoregname_Харківська область,eoregname_Херсонська область,eoregname_Хмельницька область,eoregname_Черкаська область,eoregname_Чернівецька область,eoregname_Чернігівська область,eoregname_м.Київ,mathlang_молдовська,mathlang_польська,mathlang_російська,mathlang_румунська,mathlang_угорська,mathlang_українська,mathptregname_Івано-Франківська область,mathptregname_Волинська область,mathptregname_Вінницька область,mathptregname_Дніпропетровська область,mathptregname_Донецька область,mathptregname_Житомирська область,mathptregname_Закарпатська область,mathptregname_Запорізька область,mathptregname_Київська область,mathptregname_Кіровоградська область,mathptregname_Луганська область,mathptregname_Львівська область,mathptregname_Миколаївська область,mathptregname_Одеська область,mathptregname_Полтавська область,mathptregname_Рівненська область,mathptregname_Сумська область,mathptregname_Тернопільська область,mathptregname_Харківська область,mathptregname_Херсонська область,mathptregname_Хмельницька область,mathptregname_Черкаська область,mathptregname_Чернівецька область,mathptregname_Чернігівська область,mathptregname_м.Київ
14049,32,1129,1,13,5,6480,654,1129,1252,0,1,155.0,30.0,356,223,207,18,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,

In [30]:
preprocess_test_df = preprocessor.process(
    df = test_df,
    data_type = 'test'
)

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [31]:
rf_reg = RandomForestRegressor(
    random_state=42,
    max_depth=20,
    n_estimators=500
)

In [32]:
y_train = preprocess_train_df['mathball100']
X_train = preprocess_train_df.drop(columns=['mathteststatus', 'mathball100', 'mathball'])
rf_reg.fit(X_train,y_train)

RandomForestRegressor(max_depth=20, n_estimators=500, random_state=42)

In [33]:
rf_reg.score(X_train,y_train)

0.45726157633619013

In [34]:
y_test = preprocess_test_df['mathball100']
X_test = preprocess_test_df.drop(columns=['mathteststatus', 'mathball100', 'mathball'])
rf_reg.score(X_test,y_test)

0.08350337294786414

In [35]:
from sklearn.metrics import mean_squared_error
y_pred = rf_reg.predict(X_test)
mean_squared_error(y_test, y_pred)

2445.2845344447255

In [36]:
y_pred[:5],y_test[:5]

(array([146.95953526, 145.13013299, 115.89396782, 117.40123665,
        106.13501755]),
 70862     168.0
 61092     121.0
 82820     187.0
 15192     107.0
 136368    110.0
 Name: mathball100, dtype: float64)

# Classification

In [43]:
X = df[col]

In [44]:
X.mathteststatus.unique()

array(['Не подолав поріг', 'Зараховано', 'Не з’явився', nan, 'Анульовано'],
      dtype=object)

In [45]:
X = X[(X.mathteststatus.notna()) & (X.mathteststatus!='Не з’явився')].reset_index(drop=True)
X

,sextypename,regname,areaname,tername,regtypename,tertypename,classprofilename,classlangname,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,mathtest,mathlang,mathteststatus,mathball100,mathball,mathptname,mathptregname,mathptareaname,mathpttername,age
0,чоловіча,Донецька область,м.Маріуполь,Приморський район міста,Випускник української школи поточного року,місто,Історичний,російська,"Комунальний заклад ""Маріупольська загальноосві...",середня загальноосвітня школа,Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,Департамент освіти Маріупольської міської ради...,Математика,російська,Не подолав поріг,"0,0",8.0,"Комунальний заклад ""Маріупольська загальноосві...",Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,17
1,чоловіча,Полтавська область,Миргородський район,с.Хомутець,Випускник української школи поточного року,село,Української філології,українська,"Опорний навчальний заклад ""Хомутецька загально...",середня загальноосвітня школа,Полтавська область,Миргородський район,с.Хомутець,Відділ освіти Миргородської райдержадміністрац...,Математика,українська,Зараховано,"155,0",30.0,Миргородська спеціалізована школа І-ІІІ ступен...,Полтавська область,м.Миргород,м.Миргород,17
2,чоловіча,м.Київ,м.Київ,Святошинський район міста,Випускник української школи поточного року,місто,Універсальний,українська,"Товариство з обмеженою відповідальністю ""Центр...",середня загальноосвітня школа,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,"Управління освіти, молоді та спорту Святошинсь...",Математика,українська,Зараховано,"137,0",21.0,Середня загальноосвітня школа №222 Святошинськ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,18
3,жіноча,Львівська область,м.Львів,Галицький район міста,Випускник української школи поточного року,місто,Універсальний,українська,Заклад загальної середньої освіти №87 Львівськ...,середня загальноосвітня школа,Львівська область,м.Львів. Галицький район міста,Галицький район міста,Відділ освіти Галицького та Франківського райо...,Математика,українська,Зараховано,"121,0",15.0,Середня загальноосвітня школа №84 імені Блажен...,Львівська область,м.Львів. Сихівський район міста,Сихівський район міста,17
4,жіноча,Рівненська область,Сарненський район,с.Любиковичі,Випускник української школи поточного року,село,Української філології,українська,Любиковицька загальноосвітня школа І-ІІІ ступе...,середня загальноосвітня школа,Рівненська область,Сарненський район,с.Любиковичі,"Відділ освіти, молоді та спорту Сарненської ра...",Математика,українська,Зараховано,"124,0",16.0,Сарненський педагогічний коледж Рівненського д...,Рівненська область,Сарненський район,м.Сарни,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152076,чоловіча,м.Київ,м.Київ,Святошинський район міста,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,Коледж інформаційних технологій та землевпоряд...,заклад вищої освіти,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,Міністерство освіти і науки України,Математика,українська,Зараховано,"132,0",19.0,Школа І-ІІІ ступенів № 206 імені Леся Курбаса ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,18
152077,чоловіча,Чернігівська область,Чернігівська область,м.Прилуки,Випускник української школи поточного року,місто,Інформаційно-технологічний,українська,Прилуцький заклад загальної середньої освіти І...,ліцей,Чернігівська область,м.Прилуки,м.Прилуки,Управління освіти Прилуцької міської ради,Математика,українська,Зараховано,"174,0",42.0,Прилуцький агротехнічний коледж,Чернігівська область,м.Прилуки,м.Прилуки,17
152078,жіноча,Одеська область,м.Одеса,Приморський район міста,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,Одеський технічний коледж Одеської національно...,заклад вищої освіти,Одеська область,м.Одеса. Приморський район міста,Приморський район міста,Міністерство освіти і науки Україн

In [46]:
X.mathteststatus.unique()

array(['Не подолав поріг', 'Зараховано', 'Анульовано'], dtype=object)

In [47]:
def encode_statuc_binary(x):
    if x == 'Зараховано':
        return 1
    return 0

X['mathteststatus'] = X.mathteststatus.map(encode_statuc_binary)

In [48]:
train_df,test_df = train_test_split(X, test_size=0.2, random_state=42)

In [49]:
onehotencode_col = ["sextypename", "regname", "regtypename", "eotypename", "eoregname", "mathlang", "mathptregname"]
preprocessor = Preprocessing(
    drop_strange_st_data = False,
    column_for_onehot = onehotencode_col
)

In [50]:
preprocess_train_df = preprocessor.process(
    df = train_df,
    data_type = 'train'
)
preprocess_test_df = preprocessor.process(
    df = test_df,
    data_type = 'test'
)

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [57]:
rf_cf = RandomForestClassifier(
    random_state=42,
    max_depth=10,
    n_estimators=500
)

In [58]:
y_train = preprocess_train_df['mathteststatus']
X_train = preprocess_train_df.drop(columns=['mathteststatus', 'mathball100', 'mathball'])
rf_cf.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, n_estimators=500, random_state=42)

In [59]:
rf_cf.score(X_train,y_train)

0.8750246580746975

In [60]:
y_test = preprocess_test_df['mathteststatus']
X_test = preprocess_test_df.drop(columns=['mathteststatus', 'mathball100', 'mathball'])
rf_cf.score(X_test,y_test)

0.8715849689318473

In [61]:
pred_train_y = rf_cf.predict(X_train)
pred_test_y = rf_cf.predict(X_test)

In [62]:
print('TRAIN DATA:')
print(classification_report(y_train,pred_train_y))
print('\n\n')
print('TEST DATA:')
print(classification_report(y_test,pred_test_y))

TRAIN DATA:
              precision    recall  f1-score   support

           0       0.81      0.02      0.04     15441
           1       0.88      1.00      0.93    106223

    accuracy                           0.88    121664
   macro avg       0.84      0.51      0.49    121664
weighted avg       0.87      0.88      0.82    121664




TEST DATA:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3906
           1       0.87      1.00      0.93     26511

    accuracy                           0.87     30417
   macro avg       0.44      0.50      0.47     30417
weighted avg       0.76      0.87      0.81     30417



/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo